In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
import tensorflow as tf
from tensorflow.keras import layers, models
import ast
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
from datetime import datetime
import joblib

In [2]:
df = pd.read_csv('new_landmarks/combined_dataset.csv')
# df2 = pd.read_csv('hand_landmarks_from_4500_to_5000.csv')

# df = pd.concat([df1, df2], ignore_index=True)
# df

In [3]:
df = df[df['class'] != 'to']
df.tail()

,class,landmarks
2455,z,"[[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],..."
2456,z,"[[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],..."
2457,z,"[[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],..."
2458,z,"[[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],..."
2459,z,"[[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],..."


In [4]:
def preprocess_data(df, landmarks_column="landmarks", class_column="class"):
    """
    Preprocess the dataset for model training.

    Args:
        df (pd.DataFrame): DataFrame with class labels and landmarks columns.
        landmarks_column (str): Name of the column containing landmarks.
        class_column (str): Name of the column containing class labels.

    Returns:
        pd.DataFrame: Processed DataFrame with numeric landmarks and encoded labels.
        LabelEncoder: Encoder for decoding class labels.
    """
    # Ensure landmarks are in numeric format
    df[landmarks_column] = df[landmarks_column].apply(
        lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else np.array(x)
    )

    # Encode class labels
    le = LabelEncoder()
    df["encoded_class"] = le.fit_transform(df[class_column])

    return df, le

# Apply the preprocessing
df, label_encoder = preprocess_data(df)


In [5]:
num_classes = len(label_encoder.classes_)
print(label_encoder.classes_, num_classes)

['a' 'about' 'aim' 'all' 'and' 'audio' 'b' 'barrier' 'break' 'c' 'can'
 'communication' 'creative' 'd' 'detect' 'developed' 'e' 'f' 'g' 'h'
 'have' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'our' 'p' 'project' 'q' 'r' 's'
 'sign language' 'solution' 't' 'team' 'text' 'that' 'translate' 'u' 'v'
 'w' 'what' 'x' 'y' 'you' 'z'] 49


In [6]:
def augment_landmarks(landmarks, max_noise=0.01, scale_range=(0.9, 1.1)):
    """
    Apply random augmentations to landmarks.

    Args:
        landmarks (np.ndarray): Original landmarks array.
        max_noise (float): Maximum noise to add to each coordinate.
        scale_range (tuple): Range for random scaling factors.

    Returns:
        np.ndarray: Augmented landmarks.
    """
    # Ensure landmarks are float64 for operations
    landmarks = landmarks.astype(np.float64)
    
    # Add random noise
    noise = np.random.uniform(-max_noise, max_noise, size=landmarks.shape)
    landmarks += noise

    # Apply random scaling
    scale_factor = random.uniform(*scale_range)
    landmarks *= scale_factor

    return landmarks

In [7]:
def augment_dataset(df, num_augmentations=5):
    """
    Augment the dataset by creating multiple augmented landmarks data for each row.

    Args:
        df (pd.DataFrame): DataFrame with 'class' and 'landmarks' columns.
        num_augmentations (int): Number of augmented rows to create per original row.

    Returns:
        pd.DataFrame: DataFrame with augmented data added.
    """
    augmented_rows = []
    
    for _, row in df.iterrows():
        original_landmarks = np.array(row["landmarks"])
        
        # Generate multiple augmentations for each row
        for _ in range(num_augmentations):
            augmented_landmarks = augment_landmarks(original_landmarks)
            
            # Create a new row with the same class and augmented landmarks
            new_row = {
                "class": row["class"],
                "landmarks": augmented_landmarks.tolist(),  # Convert to list for DataFrame compatibility
                "encoded_class": row.get("encoded_class", None)  # Handle missing 'encoded_class'
            }
            augmented_rows.append(new_row)
    
    # Convert augmented rows to DataFrame
    augmented_df = pd.DataFrame(augmented_rows)
    
    # Combine original and augmented data
    combined_df = pd.concat([df, augmented_df], ignore_index=True)
    return combined_df

In [8]:
df_org_augmented = augment_dataset(df, num_augmentations=3)

In [9]:
# Define timesteps, frames, and features
timesteps = 50  # Time steps
frames = 21  # Number of frames per time step
features = 3  # Number of features per frame

In [10]:
# Split the dataset into training and validation sets
df_train, df_val = train_test_split(df_org_augmented, test_size=0.2, random_state=42)

In [11]:
# Prepare X_train and y_train from the training set
X_train = np.array([
    np.array(landmark).reshape(50, 21, 6) 
    for landmark in df_train["landmarks"]
])
y_train = df_train["encoded_class"].values

# Prepare X_val and y_val from the validation set
X_val = np.array([
    np.array(landmark).reshape(50, 21, 6) 
    for landmark in df_val["landmarks"]
])
y_val = df_val["encoded_class"].values

# Verify the split and shapes
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

Training set size: 7712
Validation set size: 1928
X_train shape: (7712, 50, 21, 6)
y_train shape: (7712,)
X_val shape: (1928, 50, 21, 6)
y_val shape: (1928,)


In [12]:
# Flatten the data to 2D (samples, timesteps * features)
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_val_flattened = X_val.reshape(X_val.shape[0], -1)

# Normalize landmarks (fit_transform on the training set, and transform on validation)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_val_scaled = scaler.transform(X_val_flattened)

# Reshape the scaled data back to the original 3D shape (50, 21, 6)
X_train = X_train_scaled.reshape(X_train.shape[0], 50, 21, 6)
X_val = X_val_scaled.reshape(X_val.shape[0], 50, 21, 6)

In [13]:
# Check the shapes after reshaping
print(f"X_train shape after reshaping: {X_train.shape}")
print(f"X_val shape after reshaping: {X_val.shape}")

X_train shape after reshaping: (7712, 50, 21, 6)
X_val shape after reshaping: (1928, 50, 21, 6)


In [14]:
# Convert labels to one-hot encoding
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes)
y_val_one_hot = to_categorical(y_val, num_classes)

In [15]:
# Flatten the 3D input for ViT model (timesteps, features)
timesteps = 50  # Number of timesteps (frames)
features = 21 * 6  # Features per frame (21 landmarks * 6 values)

In [16]:
X_train_vit = X_train.reshape(-1, timesteps, features)
X_val_vit = X_val.reshape(-1, timesteps, features)

In [20]:
from mediapipe.python.solutions.hands import Hands

def build_blazepose_model(input_shape, num_classes):
    """
    Build a model for gesture classification using BlazePose-based features.

    Args:
        input_shape (tuple): Shape of the input (timesteps, features).
        num_classes (int): Number of gesture classes.

    Returns:
        keras.Model: Compiled model.
    """
    input_layer = layers.Input(shape=input_shape)

    # Process BlazePose landmark features
    x = layers.LayerNormalization()(input_layer)

    # Add LSTM to capture temporal dependencies
    x = layers.LSTM(256, return_sequences=False)(x)
    x = layers.Dropout(0.5)(x)

    # Fully connected layers for classification
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    return model

# Define input shape and number of classes
timesteps = 50  # Example value, replace with actual
features = 21 * 3 * 2  # Example: 21 landmarks with x, y, z coordinates

input_shape = (timesteps, features)
model = build_blazepose_model(input_shape, num_classes)

# Display model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 50, 126)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_1                │ (None, 50, 126)             │             252 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         392,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 49)                  │           6,321 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 431,661 (1.65 MB)

 Trainable params: 431,661 (1.65 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Set up the log directory for TensorBoard
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model
history = model.fit(
    X_train_vit,  # Reshaped training data
    y_train_one_hot,  # One-hot encoded labels for training
    validation_data=(X_val_vit, y_val_one_hot),  # Reshaped validation data and labels
    epochs=50,  # Number of epochs
    batch_size=8,  # Mini-batch size
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',  # Monitor validation loss
            patience=5,  # Stop training if no improvement for 5 epochs
            restore_best_weights=True  # Restore the best model weights
        ),
        tensorboard_callback  # TensorBoard callback
    ],
    verbose=1  # Display training progress
)

Epoch 1/50
 45/964 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.0199 - loss: 3.9951    

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.show()


In [ ]:

# Assuming 'scaler' is your scaler object
scaler_save_path = "scaler.pkl"
joblib.dump(scaler, scaler_save_path)
print(f"Scaler saved to: {scaler_save_path}")


In [ ]:
model_save_path = "my_vit_model.h5"
# Save the entire model
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model(model_save_path)

# Verify the model structure
loaded_model.summary()
